##  Projeto - Dados não estruturados:

#### Integrantes: 
- Camila Cardoso
- Fábio Barreto
- Marcelo Vega

### Identificação visual da Língua Brasileira de Sinais utilizando mediapipe e cv2

### Obejetivo: Escrever "LETSCODE"!!!

## Instruções: 

- Uma leitura será realizada a cada 1.3s
- A cada 2.7s sem novas leituras, palavra será apagada!

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import math

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def descobre_letra(lista):
    '''Recebe lista [[ponto, x,y], [ponto, x,y], ...] com a localização X e Y dos 21 pontos da mão, identifica qual letra está sendo indicada.
    Após letra ser identificada, são mostrados os pontos principais utilizados na identificação. Caso não seja identificada
    nenhuma letra, retorna Falso.
    ''' 
    # Pontas dos dedos: 4,8,12,16,20
    # Inicio dos dedos: 1,5,9,13,17
    
    # Define Variáveis:
    indicador_levantado = False
    if lista[8][2] < lista[5][2]:
        indicador_levantado = True
        
    medio_levantado = False
    if lista[12][2] < lista[9][2]:
        medio_levantado = True
        
    anelar_levantado = False
    if lista[16][2] < lista[13][2]:
        anelar_levantado = True
        
    mindinho_levantado = False
    if lista[20][2] < lista[17][2]:
        mindinho_levantado = True
        
    polegar_levantado = False
    # Alterar para distância absoluta entre 4 e 0
    ponto_4 = (lista[4][1], lista[4][2])
    ponto_0 = (lista[0][1], lista[0][2])
    ponto_5 = (lista[5][1], lista[5][2])
    if abs(math.dist(ponto_4,ponto_0)) > abs(math.dist(ponto_5,ponto_0)):
        polegar_levantado = True
     
    
    medio_com_polegar = False
    ponto_12 = (lista[12][1], lista[12][2])
    ponto_8 =  (lista[8][1], lista[8][2])
    if abs(math.dist(ponto_4, ponto_8)) <= 35:
        medio_com_polegar = True
        
        
    indicador_dobrado = False
    if lista[8][2] >= lista[6][2]-30 and lista[6][2] <= lista[5][2]+30 and indicador_levantado:
        indicador_dobrado = True
        
    medio_dobrado = False
    if lista[12][2] >= lista[10][2]-30 and lista[10][2] <= lista[9][2]+30 and medio_levantado:
        medio_dobrado = True
        
    anelar_dobrado = False
    if lista[16][2] >= lista[14][2]-30 and lista[14][2] <= lista[13][2]+30 and anelar_levantado:
        anelar_dobrado = True
    
    mindinho_dobrado = False
    if lista[20][2] >= lista[18][2]-30 and lista[18][2] <= lista[17][2]+30 and mindinho_levantado:
        mindinho_dobrado = True

        
        
    # ----
    
    # Utiliza Variáveis Para Identificar Letras: A, B, C, D, E, L, O, T, S
    if polegar_levantado and not indicador_levantado and not medio_levantado and not anelar_levantado and not mindinho_levantado:
        return 'A'
    
    elif not polegar_levantado and indicador_dobrado and medio_dobrado and anelar_dobrado:
        return 'E'
    
    elif not polegar_levantado and not indicador_dobrado and indicador_levantado and medio_levantado and anelar_levantado and mindinho_levantado and not indicador_dobrado:
        return 'B'
    
    elif not polegar_levantado and mindinho_levantado and medio_levantado and anelar_levantado and not indicador_dobrado:
        return 'T'
     
    elif not polegar_levantado and not indicador_levantado and not medio_levantado and not anelar_levantado and not mindinho_levantado:
        return 'S'
    
    elif not indicador_levantado and medio_com_polegar:
        return 'O'
    
    elif indicador_dobrado and medio_dobrado and anelar_dobrado and polegar_levantado and not medio_com_polegar:
        return 'C'
    
    elif indicador_levantado and polegar_levantado and not medio_levantado and not anelar_levantado and not mindinho_levantado:
        return 'L'
    
    elif indicador_levantado and not polegar_levantado and not medio_levantado and not anelar_levantado and not mindinho_levantado:
        return 'D'
    
    
    
    else:
        return False

In [4]:
# Instanciando classe Hands
# Para realizar os gestos de Libras, só é necessária 1 mão.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands = 1, model_complexity = 1) # equivalente a mp.solutions.hands.Hands()

# Instanciando ferramentas de desenho
mp_draw = mp.solutions.drawing_utils

# Ferramenta para customização do mp_draw
DrawingSpec = mp.solutions.drawing_utils.DrawingSpec



# Iniciando captura de video:
cap = cv2.VideoCapture(0)

tempo_antes = 0

while True:
    sucesso, img = cap.read()
    if sucesso == False:
        print('Nenhuma imagem obtida, encerrando script')
        break
    
    # BGR para RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Utilizar Hands para detectar mãos. 
    results = hands.process(img_rgb)
    
    
    h, w, c = img.shape # h: altura, w: largura, c: canais de cor
    # Se encontrar mãos, plotar pontos e arestas:
    if (results.multi_hand_landmarks):
        for hand_landmark in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmark, mp_hands.HAND_CONNECTIONS, 
                                   landmark_drawing_spec = DrawingSpec(color=(100,50,255), circle_radius=3))
            # Obtendo posições dos dedos:
            lista_pontos = []
            for _id, landmark in enumerate(hand_landmark.landmark):
                cx, cy = int(landmark.x * w), int(landmark.y * h), 
                # (ponto[0-20], x, y)
                lista_pontos.append((_id, cx, cy))
            
    
    # Cria Texto
    if (results.multi_hand_landmarks):
        # Tempo para apagar palavra
        if time.time() - tempo_antes >= 2.7:
            palavra = ''
            
        # intervalo entre cada leitura
        if time.time() - tempo_antes >= 1.3:
            letra = descobre_letra(lista_pontos)
            if letra != False:
                palavra += letra
                tempo_antes = time.time()
                
        # Imprime palavra na tela:
        cv2.putText(img, palavra, (80,450), cv2.FONT_HERSHEY_PLAIN, 3, (128,69,42), 3)        
    
    # Mostra Imagem:
    cv2.imshow("Imagem", img)
    
    # Fecha janela com a tecla S
    if cv2.waitKey(1) == ord('s'):
        break
    
cap.release()
cv2.destroyAllWindows()